### Q. Demonstrate that the thresholding logic used by perceptron is very harsh.

In [1]:
import numpy as np

# Step activation function
def step(x):
    return 1 if x >= 0 else 0

class Perceptron:
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias

    def predict(self, inputs):
        total = np.dot(self.weights, inputs) + self.bias
        return step(total)


p = Perceptron(weights=[1, 1], bias=-1)

test_inputs = [
    [0.49, 0.50],   # just below threshold
    [0.50, 0.50],   # exactly at threshold
    [0.51, 0.50]    # just above threshold
]

for x in test_inputs:
    total = np.dot(p.weights, x) + p.bias
    print(f"Input: {x}, Total: {total:.2f}, Output:", p.predict(x))

Input: [0.49, 0.5], Total: -0.01, Output: 0
Input: [0.5, 0.5], Total: 0.00, Output: 1
Input: [0.51, 0.5], Total: 0.01, Output: 1


### Q.Implement the Perceptron Learning Algorithm and study the effect of weight updates on convergence for a binary decision problem such as determining whether a user would like to watch a movie.
**Note:** 
Consider a small dataset(design your own excel csv sheet) of movie records with Boolean or real-valued features, for example:

- f1: Is actor Matt Damon present
- f2: Is the genre Thriller
- f3: Is the director Christopher Nolan
- f4: IMDb rating (scaled between 0 and 1)
The output label represents like (1) or dislike (0).
Train the perceptron model using these features and observe how weight updates influence convergence and classification performance.
Test with a sample record to show whether a perceptron properly classifies it or not.

i) Check with MP Perceptron (without weights and bias)
ii) Check with Perceptron (with weights)
iii) Check with Perceptron (with weights and bias)

In [2]:
import pandas as pd
import numpy as np

path = "/kaggle/input/movies1-csv/movies.csv"
df = pd.read_csv(path)

X = df[["f1", "f2", "f3", "f4"]].values
y = df["y"].values

print("Features (X):")
print(X)

print("\nLabels (y):")
print(y)

Features (X):
[[0.   1.   1.   0.87]
 [0.   0.   1.   0.85]
 [1.   0.   0.   0.81]
 [1.   1.   0.   0.68]
 [0.   1.   1.   0.72]
 [0.   0.   0.   0.55]]

Labels (y):
[1 1 1 0 1 0]


In [3]:
def step(x):
    return 1 if x >= 0 else 0

def mp_perceptron(x):
    return step(np.sum(x))

def perceptron_no_bias(x, weights):
    return step(np.dot(weights, x))

def perceptron_with_bias(x, weights, bias):
    return step(np.dot(weights, x) + bias)

In [4]:
class Perceptron:
    def __init__(self, learning_rate=0.1, epochs=10):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None

    # Training using Perceptron Learning Algorithm
    def fit(self, X, y):
        n_features = X.shape[1]
        self.weights = np.zeros(n_features)
        self.bias = 0

        for epoch in range(self.epochs):
            for i in range(len(X)):
                linear_output = np.dot(self.weights, X[i]) + self.bias
                y_pred = step(linear_output)
                error = y[i] - y_pred

                # Weight update
                self.weights += self.learning_rate * error * X[i]
                self.bias += self.learning_rate * error

    # Prediction
    def predict(self, X):
        predictions = []
        for x in X:
            y_pred = step(np.dot(self.weights, x) + self.bias)
            predictions.append(y_pred)
        return predictions

In [5]:
test_movie = np.array([1, 1, 0, 0.75])
weights = np.array([1,1,1,1])

print("\nTest Movie:", test_movie)


print("\n(i) MP Perceptron (no weights, no bias):")
print("Prediction:", mp_perceptron(test_movie))

print("\n(ii) Perceptron (with weights, no bias):")
print("Prediction:", perceptron_no_bias(test_movie, weights))

p = Perceptron(learning_rate=0.1, epochs=10)
p.fit(X, y)

weights = p.weights
bias = p.bias

print("\n(iii) Perceptron (with weights and bias – CLASS):")
print("Prediction:", p.predict([test_movie])[0])


Test Movie: [1.   1.   0.   0.75]

(i) MP Perceptron (no weights, no bias):
Prediction: 1

(ii) Perceptron (with weights, no bias):
Prediction: 1

(iii) Perceptron (with weights and bias – CLASS):
Prediction: 0


In [6]:
print("\nPerceptron Learning Algorithm on Dataset:")
for i in range(len(X)):
    pred = p.predict([X[i]])[0]
    print(f"Input {X[i]} -> Predicted: {pred}, Actual: {y[i]}")


Perceptron Learning Algorithm on Dataset:
Input [0.   1.   1.   0.87] -> Predicted: 1, Actual: 1
Input [0.   0.   1.   0.85] -> Predicted: 1, Actual: 1
Input [1.   0.   0.   0.81] -> Predicted: 1, Actual: 1
Input [1.   1.   0.   0.68] -> Predicted: 0, Actual: 0
Input [0.   1.   1.   0.72] -> Predicted: 1, Actual: 1
Input [0.   0.   0.   0.55] -> Predicted: 0, Actual: 0


### Q.Demonstrate the Representation Power of a Network of Perceptrons
(a) How many Boolean functions can be designed using two binary inputs?
- Ans - 2^(2^n)
(b) For each Boolean function, determine whether it is linearly separable.
- Linearly Seperable -> Learnable by a single perceptron
- Non-Linearly Seperable -> Cannot be learned by a single perceptron 
(c) Implement a single perceptron model and test whether it can correctly learn each Boolean function. (Mention how many it can't learn and why)
(d) Extend the program to estimate or analyze how the number of non-linearly separable Boolean functions increases as the number of inputs n grows.

In [7]:
import itertools
import numpy as np

# -----------------------------
# Perceptron Implementation
# -----------------------------
class Perceptron:
    def __init__(self, lr=0.1, epochs=100):
        self.lr = lr
        self.epochs = epochs

    def fit(self, X, y):
        self.w = np.zeros(X.shape[1])
        self.b = 0

        for _ in range(self.epochs):
            errors = 0
            for xi, yi in zip(X, y):
                y_pred = self.predict(xi)
                update = self.lr * (yi - y_pred)
                self.w += update * xi
                self.b += update
                errors += abs(update)
            if errors == 0:
                return True  # perfectly learned
        return False  # failed to converge

    def predict(self, x):
        return 1 if np.dot(x, self.w) + self.b >= 0 else 0


# -----------------------------
# Generate Boolean Functions
# -----------------------------
def generate_boolean_functions(n):
    inputs = list(itertools.product([0, 1], repeat=n))
    functions = list(itertools.product([0, 1], repeat=len(inputs)))
    return np.array(inputs), functions

# -----------------------------
# Test Perceptron Capability
# -----------------------------
def test_perceptron(n):
    X, functions = generate_boolean_functions(n)
    learnable = 0
    not_learnable = 0

    for f in functions:
        y = np.array(f)
        p = Perceptron()
        if p.fit(X, y):
            learnable += 1
        else:
            not_learnable += 1

    return learnable, not_learnable, len(functions)


# -----------------------------
# (c) Test for 2 Inputs
# -----------------------------
learnable, not_learnable, total = test_perceptron(2)

print("For n = 2 inputs:")
print("Total Boolean functions:", total)
print("Linearly separable (learnable):", learnable)
print("Non-linearly separable:", not_learnable)
print("Example of non-linearly separable function: XOR")


# -----------------------------
# (d) Growth Analysis
# -----------------------------
print("\nGrowth of Non-Linearly Separable Functions:")
print("n | Total Functions | Non-Linearly Separable")
print("-------------------------------------------")

for n in range(1, 4):
    _, not_learnable, total = test_perceptron(n)
    print(f"{n} | {total:15} | {not_learnable}")

For n = 2 inputs:
Total Boolean functions: 16
Linearly separable (learnable): 14
Non-linearly separable: 2
Example of non-linearly separable function: XOR

Growth of Non-Linearly Separable Functions:
n | Total Functions | Non-Linearly Separable
-------------------------------------------
1 |               4 | 0
2 |              16 | 2
3 |             256 | 152
